<a href="https://colab.research.google.com/github/spider-tronix/Dark-Sight/blob/colab_config/ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#0M NAMO NARAYANA

In [2]:
!ls

drive  ds  sample_data


In [3]:
!pip install rawpy
!pip install time
%pip install wandb -q

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [4]:
import os
os.listdir()
try:
  os.listdir().find('ds')
  first_time = False
except:
  first_time = True

In [5]:
if (first_time):
  !mkdir ds
  os.chdir('ds')
  os.listdir()

mkdir: cannot create directory ‘ds’: File exists


In [6]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
target_datasetdir = '/content/drive/My Drive/Dataset/Dataset/'
target_path = {
    'dataset': '/content/drive/My Drive/Dataset/Dataset/', 
    'save': '/content/drive/My Drive/Dataset/Dataset/results/',
    'checkpoint': '/content/drive/My Drive/Dataset/Colab/checkpoints/'
    }

In [8]:
if (first_time):
  !git clone https://github.com/spider-tronix/Dark-Sight.git
  os.chdir('./Dark-Sight')
else:
  !git pull
print(os.listdir())
!git checkout colab_config
!git status
!git log
first_time = False
print(os.listdir())

fatal: destination path 'Dark-Sight' already exists and is not an empty directory.
['Image Enhancement Baselines', 'ds.ipynb', 'DarkSight_Augmentation.ipynb', '.git', 'run', 'logs', 'PyCam Test Files', 'LICENSE', 'sid_train.py', 'Automated Dataset', 'DarkNet', 'wandb', 'Thermal Camera Libs', 'data', '.gitignore', 'Comms Over LAN']
Already on 'colab_config'
Your branch is up to date with 'origin/colab_config'.
On branch colab_config
Your branch is up to date with 'origin/colab_config'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	logs/
	wandb/

nothing added to commit but untracked files present (use "git add" to track)
commit f59df4e86124b3eb48b54b596bd2efb787834c4e (HEAD -> colab_config, origin/colab_config)
Author: Arvinth Saravanan <arvinth.saravana@gmail.com>
Date:   Sat Feb 6 12:33:52 2021 +0530

    fixed np.where in random crop

commit 6aab4b1655dac623f66b6df258c71e3b66991ac3
Author: Arvinth Saravanan <arvinth.saravana@gmail.com>
Date:   S

In [9]:
import wandb
wandb.login()

wandb: Currently logged in as: arvinth (use `wandb login --relogin` to force relogin)


True

In [ ]:
# OM NAMO NARAYANA


"""run on base dir"""


import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
import sys
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import errno

sys.path.insert(1, "./")
from DarkNet.models.sid_unet import sidUnet
from DarkNet.models.lavi_unet import laviUnet
from data.darksight_dataloader import DarkSighDataLoader
from time import time as now
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
try:
  os.listdir('./').index('logs')
except:
  !mkdir ./logs
writer = SummaryWriter('./logs')


"""Parameters"""

batch_size = 3

# set to True if thermal response included
inc_therm = False
load_weights = True

# change the directory for required model
checkpoint_dir = target_path['checkpoint']
save_dir = target_path['save']
raw_format = False
model = laviUnet(inc_therm=inc_therm, raw_format=raw_format)
plot_freq = 10
error_freq = 5
checkpoint_freq = 25
dataset_dir = target_path['dataset']

epochs = 500
lr = 1e-4


"""training code"""
os.chdir("./")
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.L1Loss()


"""creating necessary directories"""

if not raw_format:
    save_dir = save_dir + "jpg_format/"
    checkpoint_dir = checkpoint_dir + "jpg_format/"

if not inc_therm:
    save_dir = save_dir + "without_therm/"
    checkpoint_dir = checkpoint_dir + "without_therm/"

try:
    os.makedirs(save_dir)
    print(save_dir + " created")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

try:
    os.makedirs(checkpoint_dir)
    print(checkpoint_dir + " created")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

if not load_weights or os.listdir(checkpoint_dir) == []:
    if load_weights:
        print("no models saved")
    epoch_loaded = 0
else:
    checkpoint = torch.load(checkpoint_dir + os.listdir(checkpoint_dir)[-1])
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch_loaded = checkpoint["epoch"]
    print("model loaded")

trainloader = DarkSighDataLoader(
    inc_therm=inc_therm,
    raw_format=raw_format,
    dataset_dir=dataset_dir,
    batch_size=batch_size,
)


'''wandb initialization'''
example_images = []
wandb.init(project="dark-sight")
wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release
wandb.watch(model, log="all")

for epoch in range(epoch_loaded, epoch_loaded + epochs):

    running_loss = 0.0
    pre_loading = now()
    for i, data in enumerate(trainloader, 0):
        post_loading = now()
        print('time for loading a datapoint:%.3f'%((post_loading-pre_loading)/batch_size))
        inputs, gt = data
        optimizer.zero_grad()
        outputs = model(inputs)
        forward_propagation = now()
        print('forward propagation time:%.3f'%((forward_propagation-post_loading)/batch_size))
        outputs = torch.transpose(outputs, 1, 3)
        outputs = torch.transpose(outputs, 1, 2)
        loss = criterion(outputs, gt)
        loss.backward()
        optimizer.step()
        back_propagation = now()
        print('back propagation time:%.3f'%(back_propagation - forward_propagation))
        running_loss += loss.item()

        # print(
        #         "epoch: %d, iteration: %d runningloss: %.3f" % (epoch + 1, i + 1, running_loss / 2)
        #     )
        
        # writer.add_scalar('training loss',
        #                     running_loss / 1000,
        #                     epoch * len(trainloader) + i)

        wandb.log({
              "example": example_images,
              "Train Loss": running_loss/(i+1)})

        """saving results"""
        if (i+1) % batch_size == 1 and (epoch+1) % error_freq == 1:
            print(
                "epoch: %d, iteration: %d loss: %.3f" % (epoch + 1, i + 1, running_loss / 2)
            )
            running_loss = 0.0
            f, (ax1, ax2, ax3) = plt.subplots(1, 3)
            ax1.imshow(outputs.detach().numpy()[0])
            ax2.imshow(gt.detach().numpy()[0])
            ax3.imshow(
                torch.transpose(torch.transpose(inputs.detach(), 1, 3), 1, 2).numpy()[
                    0
                ][:, :, :3]
            )


            example_images.append(
                wandb.Image(
                    f, caption = "epoch:{} iteration:{}".format(epoch, i+1)
                )
            )
            
            plt.savefig(save_dir + "epoch{}_index{}.png".format(epoch + 1, i + 1))

            
            
            

        """saving checkpoints"""
        if  ( i +1 )% batch_size == 1 and ( epoch + 1) % checkpoint_freq == 1:

            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": loss,
                },
                checkpoint_dir + str(int(epoch / checkpoint_freq) + 1) + ".tar",
            )

            wandb.save('model.h5')

            print(
                "checkpoint %d_%d.tar saved."
                % (int(epoch / checkpoint_freq) + 1, i + 1)
            )


print("Finished Training")
# plt.show()


model loaded
no. of datapoints 147


Train Loss,0.42327
_runtime,25
_timestamp,1614879233
_step,0


Train Loss,▁
_runtime,▁
_timestamp,▁
_step,▁


data index 146
1977 2
data index 97
284 1
data index 39
0 1
time for loading a datapoint:1.365
forward propagation time:1.646
back propagation time:10.046
epoch: 1, iteration: 1 loss: 0.200
checkpoint 1_1.tar saved.
data index 89
1970 1
data index 145
506 1
data index 134
1112 1
time for loading a datapoint:7.964
forward propagation time:1.665
back propagation time:10.011
data index 59
784 2
data index 43
299 1
data index 33
843 1
time for loading a datapoint:14.301
forward propagation time:1.590
back propagation time:9.934
data index 139
1 1
data index 17
391 1
data index 63
0 1
time for loading a datapoint:20.564
forward propagation time:1.599
back propagation time:9.994
epoch: 1, iteration: 4 loss: 0.590
checkpoint 1_4.tar saved.
data index 31
159 1
data index 77
115 1
data index 141
1180 1
time for loading a datapoint:27.149
forward propagation time:1.672
back propagation time:9.937
data index 124
1270 1
data index 73
523 1
data index 105
747 1
time for loading a datapoint:33.924
f